# ANALYSIS OF STUDENTS' PERFORMANCE AND IDENTIFYING BIAS IN THE DATA USING CLARIFY


---

---

## Contents

1. [Overview](#Overview)
1. [Setup](#Setup)
1. [Data](#Data) 
   1. [Data Analysis](#Data-Analysis)
1. [Train](#Train)
   1. [Hyperparameter Tuning](#Hyperparameter-Tuning)
   1. [Deploy Model](#Deploy-Model)
1. [Amazon SageMaker Clarify](#Amazon-SageMaker-Clarify)
   1. [Detecting Bias](#Detecting-Bias)
   1. [Writing BiasConfig](#Writing-BiasConfig)
1. [Viewing the Bias Report](#Viewing-the-Bias-Report)
1. [Explaining Predictions](#Explaining-Predictions)
1. [Viewing the Explainability Report](#Viewing-the-Explainability-Report)
1. [Clean Up](#Clean-Up)

## Overview

Amazon SageMaker Clarify provides machine learning developers with greater visibility into their training data and models so they can identify and limit bias and explain predictions.

Biases are imbalances in the training data or the prediction behavior of the model across different groups, such as age or income bracket. Biases can result from the data or algorithm used to train your model. The field of machine learning provides an opportunity to address biases by detecting them and measuring them in your data and model.

Amazon SageMaker Clarify detects potential bias during data preparation, after model training, and in your deployed model by examining attributes you specify. For instance, you can check for bias related to age in your initial dataset or in your trained model and receive a detailed report that quantifies different types of possible bias. SageMaker Clarify also includes feature importance graphs that help you explain model predictions and produces reports which can be used to support internal presentations or to identify issues with your model that you can take steps to correct.
   
This sample notebook walks you through:

1. Key terms and concepts needed to understand SageMaker Clarify
2. Measuring the pre-training bias of a dataset and post-training bias of a model
3. Explaining the importance of the various input features on the model's decision
4. Accessing the reports through SageMaker Studio if you have an instance set up.

## Setup


Let's start by:

- Importing various Python libraries we'll need.
- Instantiate a SageMaker session for various tasks within this notebook, and get the AWS Region.
- Specifying a S3 bucket and bucket prefix to use for training and model data.
- Defining an IAM role for S3 data access, which is pulled in from the SageMaker notebook instance.


In [ ]:
from sagemaker import Session
session = Session()
# replace with your bucket and prefix
bucket = 'studentperformancebucket' 
prefix = 'clarify'
region = session.boto_region_name
# Define IAM role
from sagemaker import get_execution_role
import pandas as pd
import numpy as np
import urllib
import os
import boto3
import matplotlib.pyplot as plt
import seaborn as sns
role = get_execution_role()

---
## Data

Before proceeding further, we need to download the data from UCI repository and save to S3.

`Data Source:` http://archive.ics.uci.edu/ml/machine-learning-databases/00320/.student.zip
        
`Paper link:` http://www3.dsi.uminho.pt/pcortez/student.pdf   

In this notebook, we aim to analyse the performance of the students, using 'G3' as the label. The data was collected from two Portuguese schools. The data attributes include student grades, demographic, social and school related features and it was collected by using school reports and questionnaires. Two datasets are provided regarding the performance in two distinct subjects: Mathematics (mat) and Portuguese language (por). In [Cortez and Silva, 2008], the two datasets were modeled under binary/five-level classification and regression tasks. Important note: the target attribute G3 has a strong correlation with attributes G2 and G1. This occurs because G3 is the final year grade (issued at the 3rd period), while G1 and G2 correspond to the 1st and 2nd-period grades. It is more difficult to predict G3 without G2 and G1, but such prediction is much more useful (see paper source for more details).

`Attribute Information:`

Attributes for both student-mat.csv (Math course) and student-por.csv (Portuguese language course) datasets:
1. school - student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira)
2. sex - student's sex (binary: 'F' - female or 'M' - male)
3. age - student's age (numeric: from 15 to 22)
4. address - student's home address type (binary: 'U' - urban or 'R' - rural)
5. famsize - family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3)
6. Pstatus - parent's cohabitation status (binary: 'T' - living together or 'A' - apart)
7. Medu - mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
8. Fedu - father's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
9. Mjob - mother's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
10. Fjob - father's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), 'at_home' or 'other')
11. reason - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other')
12. guardian - student's guardian (nominal: 'mother', 'father' or 'other')
13. traveltime - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)
14. studytime - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
15. failures - number of past class failures (numeric: n if 1<=n<3, else 4)
16. schoolsup - extra educational support (binary: yes or no)
17. famsup - family educational support (binary: yes or no)
18. paid - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
19. activities - extra-curricular activities (binary: yes or no)
20. nursery - attended nursery school (binary: yes or no)
21. higher - wants to take higher education (binary: yes or no)
22. internet - Internet access at home (binary: yes or no)
23. romantic - with a romantic relationship (binary: yes or no)
24. famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
25. freetime - free time after school (numeric: from 1 - very low to 5 - very high)
26. goout - going out with friends (numeric: from 1 - very low to 5 - very high)
27. Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
28. Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
29. health - current health status (numeric: from 1 - very bad to 5 - very good)
30. absences - number of school absences (numeric: from 0 to 93)

These grades are related with the course subject, Math or Portuguese:
31. G1 - first period grade (numeric: from 0 to 20)
31. G2 - second period grade (numeric: from 0 to 20)
32. G3 - final grade (numeric: from 0 to 20, output target)
        
`For this example, we will be using the Portuguese dataset.`



In [ ]:
data_bucket = 'studentperformancebucket' # replace with your bucket name
key = 'clarify/port modified.csv' # replace with your object key

s3 = boto3.resource('s3')
s3.Bucket(data_bucket).download_file(key, 'port.csv')

data = pd.read_csv('./port.csv')

pd.set_option('display.max_rows', 20) 
data

We are dropping G1 and G2 to analyse the importance of the other features in this dataset.

In [ ]:
data = data.drop(['G1', 'G2'], axis=1)
data

---

### Data Analysis

Now let's have a look at the data

In [ ]:
data.info()

Let's check if there are any missing values in the data

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In the cells, below, we will visualise the data

In [ ]:
data['sex'].value_counts().sort_values().plot(kind='bar', title='Counts of Sex', rot=0)

There seems to be more females in the dataset as compared to males.

In [ ]:
data['G3'].value_counts().sort_values().plot(kind='bar', title='Counts of Score', rot=0)

Most students achieved a grade of '10' and only a few achieved a grade of '20'.

In [ ]:
data['absences'].value_counts().sort_values().plot(kind='bar', title='Counts of Absences', rot=0)

In [ ]:
data['internet'].value_counts().sort_values().plot(kind='bar', title='internet access', rot=0)

In [ ]:
data['failures'].value_counts().sort_values().plot(kind='bar', title='No. of failures', rot=0)

This figures below show the distribution of the positive outcomes associated with each feature.

In [ ]:
import matplotlib.pyplot as plt
from math import ceil

positive_idxs = [idx for idx, val in enumerate(data['G3']) if val > 14]

fig = plt.figure(figsize=(20,100))
cols = 2
rows = ceil(float(data.shape[1]) / cols) * 2
for i, column in enumerate(data.columns):
    ax = fig.add_subplot(rows, cols, 2 * i + 1)
    ax.set_title(column)
    if data.dtypes[column] == np.object:
        data[column][:].value_counts(sort=True).plot(kind="bar", axes=ax)
    else:
        data[column][:].hist(axes=ax)
        plt.xticks(rotation="vertical")
        
    ax = fig.add_subplot(rows, cols, 2 * i + 2)
    ax.set_title(column + " (only positive examples)")
    if data.dtypes[column] == np.object:
        data[column][positive_idxs].value_counts(sort=True).plot(kind="bar", axes=ax)
    else:
        data[column][positive_idxs].hist(axes=ax)
        plt.xticks(rotation="vertical")

plt.subplots_adjust(hspace=0.7, wspace=0.2)
    

---
## Train


To help prevent overfitting the model, we'll randomly split the data into three groups. Specifically, the model will be trained on 70% of the data. It will then be evaluated on 20% of the data to give us an estimate of the accuracy we hope to have on "new" data. As a final testing dataset, the remaining 10% will be held out until the end.

`data.sample`(frac=1, random_state=1729) returns a random sample of items from an axis of object.
The frac keyword argument specifies the fraction of rows to return in the random sample, so frac=1 means return all rows (in random order)

`len()function` returns the number of items in an object.

`random_state`, is used for initializing the internal random number generator, which will decide the splitting of data into train,validation and test indices. This is to check and validate the data when running the code multiple times. Setting random_state a fixed value will guarantee that the same sequence of random numbers is generated each time you run the code.


In [ ]:
train_data, validation_data, test_data = np.split(data.sample(frac=1, random_state=1729), [int(0.7 * len(data)), int(0.9 * len(data))])

In [ ]:
train_data

In [ ]:
test_data

Many machine learning algorithms cannot operate on label data directly. They require all input variables and output variables to be numeric.
This means that categorical data must be converted to a numerical form.

Categorical variables can be divided into two categories: Nominal (No particular order) and Ordinal (some ordered).
    
`Nominal data` simply names something without assigning it to an order in relation to other numbered objects or pieces of data. An example of nominal data might be a "pass" or "fail" classification for each student's test result. 

`Ordinal data`, unlike nominal data, involves some order; ordinal numbers stand in relation to each other in a ranked fashion. For example, suppose you receive a survey from your favourite restaurant that asks you to provide feedback on the service you received. You can rank the quality of service as "1" for poor, "2" for below average, "3" for average, "4" for very good and "5" for excellent. The data collected by this survey are examples of ordinal data. Here the numbers assigned have an order or rank; that is, a ranking of "4” is better than a ranking of “2.”

It is important to encode your data based on the above factors.

However, in this particular example, we will be treating all values as ordinal to best fit our `clarify model`. 

Hence in the next step, we will encode all categorical to numerical.

In [ ]:
from sklearn import preprocessing
def number_encode_features(df):
    result = df.copy()
    encoders = {}
    for column in result.columns:
        if result.dtypes[column] == np.object:
            encoders[column] = preprocessing.LabelEncoder()
            # print('Column:', column, result[column])
            result[column] = encoders[column].fit_transform(result[column].fillna('None'))
    return result, encoders

The data has been transformed! We will now change the first column to 'G3' which is the target variable.
Gradient boosting operates on tabular data, with the rows representing observations, one column representing the target variable or label, and the remaining columns representing features.
For CSV training, the algorithm assumes that the target variable is in the first column and that the CSV does not have a header record.

Hence, we will move 'G3' which is our target variable to the first column.

We will also save the training, validation and testing data to csv.
Amazon SageMaker's XGBoost container expects data in the libSVM or CSV data format. For this example, we'll stick to CSV. As explained above, we will remove the headers for training.

In [ ]:
training_data = pd.concat([train_data['G3'], train_data.drop(['G3'], axis=1)], axis=1)
training_data, _ = number_encode_features(training_data)
training_data.to_csv('trainP_data.csv', index=False, header=False)

validation_data = pd.concat([validation_data['G3'], validation_data.drop(['G3'], axis=1)], axis=1)
validation_data, _ = number_encode_features(validation_data)
validation_data.to_csv('validationP_data.csv', index=False, header=False)

testing_data, _ = number_encode_features(test_data)
test_features = testing_data.drop(['G3'], axis = 1)
test_target = test_data['G3']
test_features.to_csv('testP_data.csv', index=False, header=False)

In [ ]:
training_data

Now we'll copy the files to S3 for Amazon SageMaker's managed training to pickup.

In [ ]:
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

train_uri = S3Uploader.upload('trainP_data.csv', 's3://{}/{}'.format(bucket, prefix))
train_input = TrainingInput(train_uri, content_type='csv')
validation_uri = S3Uploader.upload('validationP_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_input = TrainingInput(validation_uri, content_type='csv')
test_uri = S3Uploader.upload('testP_data.csv', 's3://{}/{}'.format(bucket, prefix))

---

### Hyperparameter Tuning 

Now that we have prepared the dataset, we are ready to train models. Before we do that, one thing to note is there are algorithm settings which are called "hyperparameters" that can dramatically affect the performance of the trained models. For example, XGBoost algorithm has dozens of hyperparameters and we need to pick the right values for those hyperparameters in order to achieve the desired model training results. Since which hyperparameter setting can lead to the best result depends on the dataset as well, it is almost impossible to pick the best hyperparameter setting without searching for it, and a good search algorithm can search for the best hyperparameter setting in an automated and effective way.

We will use SageMaker hyperparameter tuning to automate the searching process effectively. Specifically, we specify a range, or a list of possible values in the case of categorical hyperparameters, for each of the hyperparameter that we plan to tune. SageMaker hyperparameter tuning will automatically launch multiple training jobs with different hyperparameter settings, evaluate results of those training jobs based on a predefined "objective metric", and select the hyperparameter settings for future attempts based on previous results. For each hyperparameter tuning job, we will give it a budget (max number of training jobs) and it will complete once that many training jobs have been executed.

In this example, we are using SageMaker Python SDK to set up and manage the hyperparameter tuning job. We first configure the training jobs the hyperparameter tuning job will launch by initiating an estimator, which includes:
* The container image for the algorithm (XGBoost)
* Configuration for the output of the training jobs
* The values of static algorithm hyperparameters, those that are not specified will be given default values
* The type and number of instances to use for the training jobs

In [ ]:
from sagemaker.image_uris import retrieve
from sagemaker.estimator import Estimator

container = retrieve('xgboost', region, version='1.2-1')
xgb = Estimator(container,
                role,
                instance_count=1,
                instance_type='ml.m4.xlarge',
                disable_profiler=True,
                sagemaker_session=session)

xgb.set_hyperparameters(eval_metric='rmse',
                        objective='reg:squarederror',
                        num_round=100,
                        rate_drop=0.3,
                        tweedie_variance_power=1.4)

We will tune five hyperparameters in this examples:
* *eta*: Step size shrinkage used in updates to prevent overfitting. After each boosting step, you can directly get the weights of new features. The eta parameter actually shrinks the feature weights to make the boosting process more conservative. 
* *alpha*: L1 regularization term on weights. Increasing this value makes models more conservative. 
* *min_child_weight*: Minimum sum of instance weight (hessian) needed in a child. If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, the building process gives up further partitioning. In linear regression models, this simply corresponds to a minimum number of instances needed in each node. The larger the algorithm, the more conservative it is. 
* *max_depth*: Maximum depth of a tree. Increasing this value makes the model more complex and likely to be overfitted. 
* *subsample*: Subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. and this will prevent overfitting. Subsampling will occur once in every boosting iteration.

You may define your own hyperparameters using these 2 links as reference:

https://xgboost.readthedocs.io/en/release_1.2.0/parameter.html#parameters-for-linear-booster-booster-gblinear
https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                        'min_child_weight': ContinuousParameter(1, 10),
                        'alpha': ContinuousParameter(0, 2),
                        'max_depth': IntegerParameter(1, 10),
                        'subsample': ContinuousParameter(0, 1)}

Now, let's define our objective metric for model validation. It is important to choose your metric accordingly. These metrics will differ according to your learning task. 

Please check the link below for more information:
https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html

In [ ]:
objective_metric_name = 'validation:rmse'

Next, we set up the tuning job with the configurations as defined below. For the sake of this example, we will set the parallel_jobs to 5 to save time. You can customise the code in order to get better results, such as increasing the total number of training jobs, etc., with the understanding that the tuning time will be increased accordingly as well.
Note: It is recommended to set the parallel jobs value to less than 10% of the total number of training jobs.

We will also need to specify the objective type for this example to 'minimize' since the default is 'maximize'. Our metric's optimization direction is 'Minimize' as stated in this guide:
https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html

In [ ]:
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=10,
                            max_parallel_jobs=5,
                            objective_type= 'Minimize',
                            )

Finally, we can now start a hyperparameter tuning job with the training and validation dataset.

Estimated time: 10 min

In [ ]:
tuner.fit({'train': train_input, 'validation': validation_input},include_cls_metadata=False)

Let's run a quick check of the hyperparameter tuning jobs status to make sure it started successfully. You may also check its status on SageMaker > Hyperparameter tuning jobs.

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

<img src="./HyperparameterTuning.PNG">

### Deploy Model

Here we will deploy the best model. 
Give a name to your model as follows:

Estimated time: 8 min

In [ ]:
xgb_predictor = tuner.deploy(initial_instance_count=1,
                             instance_type='ml.m4.xlarge',
                             model_name='Port-age-performance-clarify-model')

## Amazon SageMaker Clarify

Now that we have our model set up, we are ready to instantiate SageMaker Clarify.

In [ ]:
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.c4.xlarge',
                                                      sagemaker_session=session)

### Detecting Bias

SageMaker Clarify helps you detect possible pre- and post-training biases using a variety of metrics.

Writing DataConfig and ModelConfig
A DataConfig object communicates some basic information about data I/O to Clarify. We specify where to find the input dataset, where to store the output, the target column (label), the header names, and the dataset type.

Similarly, the ModelConfig object communicates information about the trained model and ModelPredictedLabelConfig provides information on the format of our predictions.

Note: To avoid additional traffic to your production models, SageMaker Clarify sets up and tears down a dedicated endpoint when processing. ModelConfig specifies your preferred instance type and instance count used to run your model on during Clarify's processing.

Here we specifiy our model's name again.

For regression task, the model returns a score. Hence, we do not need to specify anything for ModelPredictedLabelConfig. 

In [ ]:
bias_report_output_path = 's3://{}/{}/clarify-bias-P-age'.format(bucket, prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='G3',
                                      headers=training_data.columns.to_list(),
                                      dataset_type='text/csv')

model_config = clarify.ModelConfig(model_name='Port-age-performance-clarify-model',
                                   instance_type='ml.c5.2xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

predictions_config = clarify.ModelPredictedLabelConfig()

### Writing BiasConfig

SageMaker Clarify also needs information on what the sensitive columns (facets) are, what the sensitive features (facet_values_or_threshold) may be, and what the desirable outcomes are (label_values_or_threshold). Clarify can handle both categorical and continuous data for facet_values_or_threshold and for label_values_or_threshold. In this case we are using categorical data.

We specify this information in the BiasConfig API. The positive outcome is achieving a grade of 15 or more, sex is a sensitive category, and Female respondents are the sensitive group.
You may include 'facet_values_or_threshold' and set it to 0 to examine bias in sex, particularly, female. However, for this example, we will be examining both.
The 'group_name'indicates a column to be used for the bias metric, ‘Conditional Demographic Disparity in Labels - CDDL’ or ‘Conditional Demographic Disparity in Predicted Labels - CDDPL’. 

`Note: Female has been encoded as 0 and male as 1.`

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[15,16,17,18,19,20],
                                facet_name= 'sex',
                                #facet_values_or_threshold=[0]
                                )
                    

<img src="./port-sex.gif">

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[15,16,17,18,19,20],
                                facet_name= 'age', 
                                #facet_values_or_threshold=[22]
                                )

<img src="./port-age.gif">

Pre-training Bias
Bias can be present in the data before any model training occurs. Inspecting data for bias before training begins can help detect any data collection gaps, inform your feature engineering, and help us understand what societal biases the data may reflect.

Computing pre-training bias metrics does not require a trained model.

Post-training Bias
Computing post-training bias metrics does require a trained model.

Unbiased training data (as determined by concepts of fairness measured by bias metric) may still result in biased model predictions after training. Whether this occurs depends on several factors including hyperparameter choices.

You can run these options separately with run_pre_training_bias() and run_post_training_bias() or at the same time with run_bias() as shown below.

In [ ]:
clarify_processor.run_pre_training_bias(data_config=bias_data_config,
                                        data_bias_config=bias_config,
                                        methods='all',
                                        job_name='pre-trained-P')

In [ ]:
clarify_processor.run_post_training_bias(data_config=bias_data_config,
                                         data_bias_config=bias_config,
                                         model_config=model_config,
                                         model_predicted_label_config=predictions_config,
                                         methods='all',
                                         job_name='post-trained-P')

Estimated time: 13 min

In [ ]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all',
                           job_name='Clarify-Bias-P-age-1')


## Viewing the Bias Report

In Studio, you can view the results under the experiments tab.

Each bias metric has detailed explanations with examples that you can explore.

You could also summarise the results in a handy table!
Image

If you're not a Studio user yet, you can access the bias report in pdf, html and ipynb formats in the following S3 bucket:


In [ ]:
bias_report_output_path

## Explaining Predictions
There are expanding business needs and legislative regulations that require explainations of _why_ a model makes the decision it did. SageMaker Clarify uses SHAP to explain the contribution each input feature makes to the final decision.
For each input example in the ‘s3_data_input_path’ the SHAP algorithm determines feature importance, by creating ‘num_samples’ copies of the example with a subset of features replaced with values from the ‘baseline’. Model inference is run to see how the prediction changes with the replaced features. If the model output returns multiple scores, importance is computed for each of them. Across examples, feature importance is aggregated using ‘agg_method’.

In [ ]:
shap_config = clarify.SHAPConfig(baseline=[test_features.iloc[0].values.tolist()],
                                 num_samples=50,
                                 agg_method='mean_abs')

explainability_output_path = 's3://{}/{}/clarify-explainability-P'.format(bucket, prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='G3',
                                headers=data.columns.to_list(),
                                dataset_type='text/csv')

Estimated Time: 13 min

In [ ]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config,
                                     job_name='Clarify-Explainability-P2')

## Viewing the Explainability Report
As with the bias report, you can view the explainability report in Studio under the experiments tab

The Model Insights tab contains direct links to the report and model insights.

If you're not a Studio user yet, as with the Bias Report, you can access this report at the following S3 bucket.

In [ ]:
explainability_output_path

<img src="./port-explain.png">

## Clean Up
Finally, don't forget to clean up the resources we set up and used for this demo!

In [ ]:
xgb_predictor.delete_endpoint()